# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [7]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.add("HiGHS")
Pkg.add("DataFrames")
Pkg.add("GraphRecipes")
Pkg.add("Plots")
Pkg.add("Measures")
Pkg.add("MarkdownTables")

  Activating project at `c:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5`
Precompiling project...
  ✓ Latexify → DataFramesExt
  ✓ Plots → UnitfulExt
  2 dependencies successfully precompiled in 114 seconds. 230 already precompiled.
   Resolving package versions...
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Project.toml`
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Project.toml`
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Project.toml`
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\FM's Laptop\Downloads\College\BEE 4750\HW\HW5\Project.toml`
  No Changes to `C:\Use

In [8]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [ ]:
# Define the model
model = Model(HiGHS.Optimizer)

# Parameters where the order is [LF,MRF,WTE]
city_waste = [100, 90, 120] # Mg/day
facility_capacities = [200, 350, 210] # Mg
fixed_costs = [2000, 1500, 2500] #$/Day
tipping_costs = [50, 7, 60] # $/Mg
recycling_cost = 40 # $/Mg
transport_cost = 1.5 # $/Mg-km
distances = [
    [5, 30, 15], # city 1
    [15, 25, 10], # city 2
    [13, 45, 20] # city 3
] # City to facilities

facility_to_facility_distances = [
    [0, 32, 18], # LF to other facilities
    [32, 0, 15], # MRF to other facilities
    [18, 15, 0]  # WTE to other facilities
]

recycling_rate = 0.4
ash_fractions = [0.16, 0.14] # non-recycled, recycled

# Decision variables
@variable(model, x[1:3, 1:3] >= 0) # Waste sent from city i to facility j
@variable(model, r[1:3, 1:3] >= 0) # Residual waste from facility j to k
@variable(model, use_facility[1:3], Bin) # Whether a facility is used

# Objective: Minimize total cost
@objective(
    model, Min,
    sum(transport_cost * distances[i, j] * x[i, j] for i in 1:3, j in 1:3) +
    sum(residual_transport_cost * facility_to_facility_distances[j, k] * r[j, k] for j in 1:3, k in 1:3) +
    sum(fixed_costs[j] * use_facility[j] for j in 1:3) +
    sum(tipping_costs[j] * x[i, j] for i in 1:3, j in 1:3) +
    recycling_cost * recycling_rate * sum(x[i, 2] for i in 1:3)
)

# Constraints
# Balance waste generated and disposed
for i in 1:3
    @constraint(model, sum(x[i, j] for j in 1:3) == city_waste[i])
end

# Capacity constraints for each facility
for j in 1:3
    @constraint(
        model,
        sum(x[i, j] for i in 1:3) + sum(r[k, j] for k in 1:3) <= facility_capacities[j] * use_facility[j]
    )
end

# Residual ash constraint for MRF
@constraint(
    model,
    sum(x[i, 2] for i in 1:3) * recycling_rate * ash_fractions[2] +
    sum(x[i, 2] for i in 1:3) * (1 - recycling_rate) * ash_fractions[1] +
    sum(r[k, 2] for k in 1:3) <= facility_capacities[2]
)
# Solve the model
optimize!(model)

# Output results
println("Optimal Cost: ", objective_value(model))
for j in 1:3
    println("Facility $j Used: ", value(use_facility[j]))
end
for i in 1:3, j in 1:3
    println("Waste from City $i to Facility $j: ", value(x[i, j]))
end
for j in 1:3, k in 1:3
    println("Residual Waste from Facility $j to Facility $k: ", value(r[j, k]))
end

BoundsError: BoundsError: attempt to access 3-element Vector{Vector{Int64}} at index [1, 2]

## References

List any external references consulted, including classmates.